In [ ]:
import math,os,cv2,random
from tqdm import tqdm
import pandas as pd
import numpy as np

#torch
import torch
import torch.nn as nn
from torch.nn import Parameter
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim import Adam, lr_scheduler

#transformers
import transformers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup,get_cosine_schedule_with_warmup
from transformers import get_cosine_with_hard_restarts_schedule_with_warmup

# exclude warnings
import warnings
warnings.simplefilter('ignore')

In [ ]:
GEN_BACK_TR = True

GEN_UPSAMPLE = False

GEN_EN_ONLY = False
#nlp augmentation
#!pip install googletrans
!pip install googletrans==4.0.0-rc1
from googletrans import Translator

In [ ]:
trainpath = "../input/commonlitreadabilityprize/train.csv"
df = pd.read_csv(trainpath)
train_df = df[['id','excerpt','target','standard_error']].copy()
train_df.head()

In [ ]:
def back_translate(sequence, lang, org_lang='en'):
    
    #instantiate translator
    translator = Translator()

    #store original language so we can convert back
    #org_lang = translator.detect(sequence).lang
    #print(org_lang)
    
    #randomly choose language to translate sequence to  
    #random_lang = np.random.choice([lang for lang in languages if lang is not org_lang])
    
    #if org_lang == 'en':
    #translate to new language and back to original
    translated = translator.translate(sequence, dest = lang).text
    #translate back to original language
    translated_back = translator.translate(translated, dest = 'en').text
    
        #apply with certain probability
        #if np.random.uniform(0, 1) <= PROB:
    output_sequence = translated_back
        #else:
        #output_sequence = sequence
            
    #if detected language not in our list of languages, do nothing
    #else:
        #output_sequence = sequence
    
    return output_sequence

In [ ]:
languages = ['en', 'fr', 'th', 'tr', 'ur', 'ru', 'bg', 'de', 'ar', 'zh-cn', 'hi', 'sw', 'vi', 'es', 'el']

languages = ['fr', 'th', 'tr', 'ur', 'ru', 'bg', 'de', 'ar']

In [ ]:
#lang = languages[1]
#train_df[lang] = ''
#print(train_df)

In [ ]:
for lang in languages:
    train_df[lang] = ''
    for index,row in df.iterrows():
        train_df[lang].loc[index] = back_translate(row.excerpt, lang)
        if index%10==0:
            print(index)

In [ ]:
train_df

In [ ]:
train_df.to_csv("train.csv")